# Imports

In [1]:
import os
import pandas as pd
from jigsawstack import JigsawStack
from dotenv import load_dotenv, find_dotenv

In [2]:
from sqlalchemy import create_engine

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
assert not os.getenv("JIGSAWSTACK_API") is None

# Setup

## APIs

In [5]:
jigsawstack = JigsawStack(
    api_key = os.getenv("JIGSAWSTACK_API")
)

## data

In [6]:
product_table_df = pd.read_csv('data/tables/Product Table.csv')
invoice_table_df = pd.read_csv('data/tables/Invoice Table.csv')
invoice_item_df = pd.read_csv('data/tables/Invoice Item.csv')
outlet_df = pd.read_csv('data/tables/Outlet Table.csv')
warehouse_df = pd.read_csv('data/tables/Warehouse Table.csv')
address_df = pd.read_csv('data/tables/Address Table.csv')

## SQLlite

In [7]:
engine = create_engine('sqlite:///my_database.db')

In [8]:
product_table_df.to_sql('datachat_product', engine, index=False, if_exists='replace')
invoice_table_df.to_sql('datachat_invoice', engine, index=False, if_exists='replace')
invoice_item_df.to_sql('datachat_invoiceitem', engine, index=False, if_exists='replace')
outlet_df.to_sql('datachat_outlet', engine, index=False, if_exists='replace')
warehouse_df.to_sql('datachat_warehouse', engine, index=False, if_exists='replace')
address_df.to_sql('datachat_address', engine, index=False, if_exists='replace')

5

In [9]:
product_table_schema_sql = pd.io.sql.get_schema(product_table_df, 'datachat_product', con=engine)
invoice_table_schema_sql = pd.io.sql.get_schema(invoice_table_df, 'datachat_invoice', con=engine)
invoice_item_schema_sql = pd.io.sql.get_schema(invoice_item_df, 'datachat_invoiceitem', con=engine)
outlet_schema_sql = pd.io.sql.get_schema(outlet_df, 'datachat_outlet', con=engine)
warehouse_schema_sql = pd.io.sql.get_schema(warehouse_df, 'datachat_warehouse', con=engine)
address_schema_sql = pd.io.sql.get_schema(address_df, 'datachat_address', con=engine)

In [10]:
schema_sql = '\n'.join([
    product_table_schema_sql,
    invoice_table_schema_sql,
    invoice_item_schema_sql,
    outlet_schema_sql,
    warehouse_schema_sql,
address_schema_sql])
print(schema_sql)


CREATE TABLE datachat_product (
	id BIGINT, 
	product_code TEXT, 
	name TEXT
)



CREATE TABLE datachat_invoice (
	invoice_id BIGINT, 
	timestamp TEXT, 
	subtotal FLOAT, 
	gst FLOAT, 
	total BIGINT, 
	origin_address_id BIGINT, 
	origin_address_type BIGINT
)



CREATE TABLE datachat_invoiceitem (
	id BIGINT, 
	invoice_id BIGINT, 
	product_id BIGINT, 
	"quantity " BIGINT, 
	price BIGINT, 
	line_total BIGINT
)



CREATE TABLE datachat_outlet (
	id BIGINT, 
	name TEXT, 
	address_id BIGINT
)



CREATE TABLE datachat_warehouse (
	id BIGINT, 
	name TEXT, 
	address_id BIGINT
)



CREATE TABLE datachat_address (
	id BIGINT, 
	street TEXT, 
	"postal code" BIGINT
)




# Unit Testings

## Prompt 1

In [11]:
user_query = "I want to find out how D0001 and R0001 perform across outlets 1 to 3. Please output the results in a table where column headers are outlet names and row headers are product names"
print(user_query)

I want to find out how D0001 and R0001 perform across outlets 1 to 3. Please output the results in a table where column headers are outlet names and row headers are product names


In [12]:
expected_response = """
SELECT 
    p.product_code,
    SUM(CASE WHEN o.name = "Outlet 1" THEN ii.line_total ELSE 0 END) AS "Outlet 1" ,
    SUM(CASE WHEN o.name = "Outlet 2" THEN ii.line_total ELSE 0 END) AS "Outlet 2" ,
    SUM(CASE WHEN o.name = "Outlet 3" THEN ii.line_total ELSE 0 END) AS "Outlet 3"
FROM datachat_outlet o
    JOIN datachat_invoice i ON i.origin_address_id = o.address_id
        JOIN datachat_invoiceitem ii ON ii.invoice_id = i.invoice_id
            JOIN datachat_product p ON p.id = ii.product_id
WHERE p.product_code IN ('D0001', 'R0001')
GROUP BY p.product_code;
"""
print(expected_response)


SELECT 
    p.product_code,
    SUM(CASE WHEN o.name = "Outlet 1" THEN ii.line_total ELSE 0 END) AS "Outlet 1" ,
    SUM(CASE WHEN o.name = "Outlet 2" THEN ii.line_total ELSE 0 END) AS "Outlet 2" ,
    SUM(CASE WHEN o.name = "Outlet 3" THEN ii.line_total ELSE 0 END) AS "Outlet 3"
FROM datachat_outlet o
    JOIN datachat_invoice i ON i.origin_address_id = o.address_id
        JOIN datachat_invoiceitem ii ON ii.invoice_id = i.invoice_id
            JOIN datachat_product p ON p.id = ii.product_id
WHERE p.product_code IN ('D0001', 'R0001')
GROUP BY p.product_code;



In [13]:
result_df = pd.read_sql_query(expected_response, engine)
result_df

,product_code,Outlet 1,Outlet 2,Outlet 3
0,D0001,200,400,1200
1,R0001,200,1000,600


### 0 shot

In [14]:
print(user_query)

I want to find out how D0001 and R0001 perform across outlets 1 to 3. Please output the results in a table where column headers are outlet names and row headers are product names


In [15]:
params = {
    "prompt": user_query,
    "sql_schema": schema_sql,
    'database': 'sqlite',
}
result = jigsawstack.text_to_sql(params)

In [16]:
sql_query = result['sql']
print(sql_query)

SELECT p.name AS product_name, o1.name AS outlet_1, o2.name AS outlet_2, o3.name AS outlet_3 FROM datachat_product p LEFT JOIN datachat_invoiceitem ii ON p.id = ii.product_id LEFT JOIN datachat_invoice i ON ii.invoice_id = i.invoice_id LEFT JOIN datachat_outlet o1 ON i.origin_address_id = o1.address_id AND o1.id = 1 LEFT JOIN datachat_outlet o2 ON i.origin_address_id = o2.address_id AND o2.id = 2 LEFT JOIN datachat_outlet o3 ON i.origin_address_id = o3.address_id AND o3.id = 3 WHERE p.product_code IN ('D0001', 'R0001') GROUP BY p.name


In [17]:
result_df = pd.read_sql_query(sql_query, engine)
result_df

,product_name,outlet_1,outlet_2,outlet_3
0,Helicopter Drone,Outlet 1,None,None
1,Robot Dolly,Outlet 1,None,None


### Few shot

In [18]:
user_query_v1 = "I want to find out how D0001 and T0001 perform across outlets 1 to 2. Please output the results in a table where column headers are outlet names and row headers are product names"
print(user_query_v1)

I want to find out how D0001 and T0001 perform across outlets 1 to 2. Please output the results in a table where column headers are outlet names and row headers are product names


In [19]:
expected_response_v1 = """
SELECT 
    p.product_code,
    SUM(CASE WHEN o.name = "Outlet 1" THEN ii.line_total ELSE 0 END) AS "Outlet 1" ,
    SUM(CASE WHEN o.name = "Outlet 2" THEN ii.line_total ELSE 0 END) AS "Outlet 2"
FROM datachat_outlet o
    JOIN datachat_invoice i ON i.origin_address_id = o.address_id
        JOIN datachat_invoiceitem ii ON ii.invoice_id = i.invoice_id
            JOIN datachat_product p ON p.id = ii.product_id
WHERE p.product_code IN ('D0001', 'T0001')
GROUP BY p.product_code;
"""
print(expected_response_v1)


SELECT 
    p.product_code,
    SUM(CASE WHEN o.name = "Outlet 1" THEN ii.line_total ELSE 0 END) AS "Outlet 1" ,
    SUM(CASE WHEN o.name = "Outlet 2" THEN ii.line_total ELSE 0 END) AS "Outlet 2"
FROM datachat_outlet o
    JOIN datachat_invoice i ON i.origin_address_id = o.address_id
        JOIN datachat_invoiceitem ii ON ii.invoice_id = i.invoice_id
            JOIN datachat_product p ON p.id = ii.product_id
WHERE p.product_code IN ('D0001', 'T0001')
GROUP BY p.product_code;



In [20]:
user_query_fewshot = '\n'.join([
    user_query_v1,
    expected_response_v1,
    user_query,
])
print(user_query_fewshot)

I want to find out how D0001 and T0001 perform across outlets 1 to 2. Please output the results in a table where column headers are outlet names and row headers are product names

SELECT 
    p.product_code,
    SUM(CASE WHEN o.name = "Outlet 1" THEN ii.line_total ELSE 0 END) AS "Outlet 1" ,
    SUM(CASE WHEN o.name = "Outlet 2" THEN ii.line_total ELSE 0 END) AS "Outlet 2"
FROM datachat_outlet o
    JOIN datachat_invoice i ON i.origin_address_id = o.address_id
        JOIN datachat_invoiceitem ii ON ii.invoice_id = i.invoice_id
            JOIN datachat_product p ON p.id = ii.product_id
WHERE p.product_code IN ('D0001', 'T0001')
GROUP BY p.product_code;

I want to find out how D0001 and R0001 perform across outlets 1 to 3. Please output the results in a table where column headers are outlet names and row headers are product names


In [21]:
params = {
    "prompt": user_query_fewshot,
    "sql_schema": schema_sql,
    'database': 'sqlite',
}
result = jigsawstack.text_to_sql(params)

In [22]:
sql_query = result['sql']
print(sql_query)

SELECT    p.product_code,   SUM(CASE WHEN o.name = "Outlet 1" THEN ii.line_total ELSE 0 END) AS "Outlet 1",   SUM(CASE WHEN o.name = "Outlet 2" THEN ii.line_total ELSE 0 END) AS "Outlet 2",   SUM(CASE WHEN o.name = "Outlet 3" THEN ii.line_total ELSE 0 END) AS "Outlet 3" FROM datachat_outlet o   JOIN datachat_invoice i ON i.origin_address_id = o.address_id       JOIN datachat_invoiceitem ii ON ii.invoice_id = i.invoice_id           JOIN datachat_product p ON p.id = ii.product_id WHERE p.product_code IN ('D0001', 'R0001') GROUP BY p.product_code;


In [23]:
result_df = pd.read_sql_query(sql_query, engine)
result_df

,product_code,Outlet 1,Outlet 2,Outlet 3
0,D0001,200,400,1200
1,R0001,200,1000,600


## Prompt 2

In [24]:
user_query = "I want to find out what percentage of my total sales come from warehouse sales"

In [25]:
expected_response = """
SELECT 
  (SUM(i.total) * 100.0 / (SELECT SUM(total) FROM datachat_invoice)) AS warehouse_sales_percentage
FROM datachat_invoice i
JOIN datachat_warehouse w 
    ON i.origin_address_id = w.address_id
WHERE i.origin_address_type = 2;
"""
print(expected_response)


SELECT 
  (SUM(i.total) * 100.0 / (SELECT SUM(total) FROM datachat_invoice)) AS warehouse_sales_percentage
FROM datachat_invoice i
JOIN datachat_warehouse w 
    ON i.origin_address_id = w.address_id
WHERE i.origin_address_type = 2;



In [26]:
result_df = pd.read_sql_query(expected_response, engine)
result_df

,warehouse_sales_percentage
0,40.47619


### 0 shot

In [27]:
print(user_query)

I want to find out what percentage of my total sales come from warehouse sales


In [28]:
params = {
    "prompt": user_query,
    "sql_schema": schema_sql,
    'database': 'sqlite',
}
result = jigsawstack.text_to_sql(params)

In [29]:
sql_query = result['sql']
print(sql_query)

SELECT json_object('warehouse_sales_percentage', (SUM(CASE WHEN di.origin_address_type = 2 THEN di.total ELSE 0 END) * 100.0 / SUM(di.total))) FROM datachat_invoice di


In [30]:
result_df = pd.read_sql_query(sql_query, engine)
result_df

,"json_object('warehouse_sales_percentage', (SUM(CASE WHEN di.origin_address_type = 2 THEN di.total ELSE 0 END) * 100.0 / SUM(di.total)))"
0,"{""warehouse_sales_percentage"":40.4761904761905}"


### Few shot

In [31]:
user_query_v2 = "I want to find out what percentage of my total sales come from outlet sales"

In [32]:
expected_response_v2 = """
SELECT 
  (SUM(i.total) * 100.0 / (SELECT SUM(total) FROM datachat_invoice)) AS outlet_sales_percentage
FROM datachat_invoice i
JOIN datachat_outlet w 
    ON i.origin_address_id = w.address_id
WHERE i.origin_address_type = 1;
"""
print(expected_response_v2)


SELECT 
  (SUM(i.total) * 100.0 / (SELECT SUM(total) FROM datachat_invoice)) AS outlet_sales_percentage
FROM datachat_invoice i
JOIN datachat_outlet w 
    ON i.origin_address_id = w.address_id
WHERE i.origin_address_type = 1;



In [33]:
result_df = pd.read_sql_query(expected_response_v2, engine)
result_df

,outlet_sales_percentage
0,59.52381


In [34]:
user_query_fewshot = '\n'.join([
    user_query_v2,
    expected_response_v2,
    user_query,
])
print(user_query_fewshot)

I want to find out what percentage of my total sales come from outlet sales

SELECT 
  (SUM(i.total) * 100.0 / (SELECT SUM(total) FROM datachat_invoice)) AS outlet_sales_percentage
FROM datachat_invoice i
JOIN datachat_outlet w 
    ON i.origin_address_id = w.address_id
WHERE i.origin_address_type = 1;

I want to find out what percentage of my total sales come from warehouse sales


In [35]:
params = {
    "prompt": user_query_fewshot,
    "sql_schema": schema_sql,
    'database': 'sqlite',
}
result = jigsawstack.text_to_sql(params)

In [36]:
sql_query = result['sql']
print(sql_query)

SELECT (SUM(i.total) * 100.0 / (SELECT SUM(total) FROM datachat_invoice)) AS warehouse_sales_percentage FROM datachat_invoice i JOIN datachat_warehouse w ON i.origin_address_id = w.address_id WHERE i.origin_address_type = 2;


In [37]:
result_df = pd.read_sql_query(sql_query, engine)
result_df

,warehouse_sales_percentage
0,40.47619


## Prompt 3

In [38]:
user_query = "Please track all invoices by location and create a bar chart in excel where x axis is outlet or warehouse name and y axis is sales amount"

# Demo